In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 100
test_batch = 200
lr = 0.04
schedule = [75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/aug_real' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.1
blog_sig = 0.5
jpg_prob = 0.1

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(target_dir, '100_shot_style2_onlyreal')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.RandomCrop(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.CenterCrop(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


# Loss

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [13]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [14]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

In [16]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    if epoch % 20 == 0:
        test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : teacher_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.040000
1/4 | Loss:2.2377 | MainLoss:2.2184 | SPLoss:0.0000 | CLSLoss:1.9344 | top1:66.0000 | AUROC:0.7571
4/4 | Loss:1.0398 | MainLoss:0.9830 | SPLoss:0.3865 | CLSLoss:1.8082 | top1:67.0000 | AUROC:0.8525
39/4 | Loss:0.8597 | MainLoss:0.6956 | SPLoss:1.4785 | CLSLoss:1.6215 | top1:50.2051 | AUROC:0.4987
161/4 | Loss:0.7633 | MainLoss:0.5993 | SPLoss:1.4785 | CLSLoss:1.6215 | top1:97.3645 | AUROC:0.9987

Epoch: [2 | 4000] LR: 0.068000
1/4 | Loss:0.8231 | MainLoss:0.6591 | SPLoss:1.4785 | CLSLoss:1.6215 | top1:63.0000 | AUROC:0.8378
4/4 | Loss:0.9986 | MainLoss:0.6742 | SPLoss:3.0957 | CLSLoss:1.4853 | top1:54.0000 | AUROC:0.8010

Epoch: [3 | 4000] LR: 0.096000
1/4 | Loss:1.2804 | MainLoss:0.6774 | SPLoss:5.9015 | CLSLoss:1.2907 | top1:69.0000 | AUROC:0.7332
4/4 | Loss:1.5316 | MainLoss:0.7073 | SPLoss:8.1265 | CLSLoss:1.1642 | top1:54.2500 | AUROC:0.7010

Epoch: [4 | 4000] LR: 0.124000
1/4 | Loss:1.8396 | MainLoss:0.6726 | SPLoss:11.5732 | CLSLoss:0.9684 | top1:

1/4 | Loss:7.2858 | MainLoss:0.6321 | SPLoss:66.5021 | CLSLoss:0.3517 | top1:65.0000 | AUROC:0.7827
4/4 | Loss:11.1734 | MainLoss:0.6336 | SPLoss:105.3603 | CLSLoss:0.3746 | top1:66.0000 | AUROC:0.7747

Epoch: [35 | 4000] LR: 0.319974
1/4 | Loss:39.4231 | MainLoss:0.5888 | SPLoss:388.3092 | CLSLoss:0.3380 | top1:72.0000 | AUROC:0.7897
4/4 | Loss:74.7012 | MainLoss:0.6520 | SPLoss:740.4606 | CLSLoss:0.3074 | top1:61.5000 | AUROC:0.7044

Epoch: [36 | 4000] LR: 0.319972
1/4 | Loss:120.4906 | MainLoss:0.6825 | SPLoss:1198.0565 | CLSLoss:0.2451 | top1:58.0000 | AUROC:0.6558
4/4 | Loss:120.2173 | MainLoss:0.6727 | SPLoss:1195.4249 | CLSLoss:0.2168 | top1:57.5000 | AUROC:0.6767

Epoch: [37 | 4000] LR: 0.319969
1/4 | Loss:98.0847 | MainLoss:0.6726 | SPLoss:974.1041 | CLSLoss:0.1781 | top1:57.0000 | AUROC:0.6110
4/4 | Loss:71.8998 | MainLoss:0.6644 | SPLoss:712.3356 | CLSLoss:0.1770 | top1:59.5000 | AUROC:0.6562

Epoch: [38 | 4000] LR: 0.319967
1/4 | Loss:31.8783 | MainLoss:0.6187 | SPLoss:312.

1/4 | Loss:144.0120 | MainLoss:0.5960 | SPLoss:1434.1357 | CLSLoss:0.2430 | top1:72.0000 | AUROC:0.7904
4/4 | Loss:140.4501 | MainLoss:0.6176 | SPLoss:1398.2976 | CLSLoss:0.2725 | top1:66.5000 | AUROC:0.7457

Epoch: [68 | 4000] LR: 0.319845
1/4 | Loss:110.1405 | MainLoss:0.6285 | SPLoss:1095.0898 | CLSLoss:0.2967 | top1:66.0000 | AUROC:0.7222
4/4 | Loss:78.4047 | MainLoss:0.6588 | SPLoss:777.4313 | CLSLoss:0.2749 | top1:62.0000 | AUROC:0.6607

Epoch: [69 | 4000] LR: 0.319840
1/4 | Loss:30.5687 | MainLoss:0.6342 | SPLoss:299.3187 | CLSLoss:0.2624 | top1:64.0000 | AUROC:0.6846
4/4 | Loss:17.1211 | MainLoss:0.6117 | SPLoss:165.0644 | CLSLoss:0.2989 | top1:65.7500 | AUROC:0.7250

Epoch: [70 | 4000] LR: 0.319834
1/4 | Loss:10.0380 | MainLoss:0.6575 | SPLoss:93.7670 | CLSLoss:0.3790 | top1:64.0000 | AUROC:0.6916
4/4 | Loss:22.1876 | MainLoss:0.7081 | SPLoss:214.7616 | CLSLoss:0.3363 | top1:56.0000 | AUROC:0.5266

Epoch: [71 | 4000] LR: 0.319828
1/4 | Loss:46.6281 | MainLoss:0.7332 | SPLoss:4

1/4 | Loss:3.1964 | MainLoss:0.6398 | SPLoss:25.5381 | CLSLoss:0.2787 | top1:63.0000 | AUROC:0.6863
4/4 | Loss:6.7090 | MainLoss:0.5906 | SPLoss:61.1565 | CLSLoss:0.2768 | top1:63.7500 | AUROC:0.7305
39/4 | Loss:17.1151 | MainLoss:0.7392 | SPLoss:163.7313 | CLSLoss:0.2765 | top1:48.1410 | AUROC:0.4809
161/4 | Loss:16.7215 | MainLoss:0.3456 | SPLoss:163.7315 | CLSLoss:0.2765 | top1:90.5950 | AUROC:0.9742

Epoch: [102 | 4000] LR: 0.031960
1/4 | Loss:16.9139 | MainLoss:0.5380 | SPLoss:163.7313 | CLSLoss:0.2765 | top1:68.0000 | AUROC:0.8145
4/4 | Loss:25.3926 | MainLoss:0.5824 | SPLoss:248.0745 | CLSLoss:0.2770 | top1:64.5000 | AUROC:0.7698

Epoch: [103 | 4000] LR: 0.031959
1/4 | Loss:39.0830 | MainLoss:0.5355 | SPLoss:385.4472 | CLSLoss:0.2747 | top1:69.0000 | AUROC:0.8016
4/4 | Loss:45.8888 | MainLoss:0.5310 | SPLoss:453.5506 | CLSLoss:0.2772 | top1:70.0000 | AUROC:0.8059

Epoch: [104 | 4000] LR: 0.031958
1/4 | Loss:55.4721 | MainLoss:0.5776 | SPLoss:548.9164 | CLSLoss:0.2810 | top1:64.0

1/4 | Loss:22.2641 | MainLoss:0.4495 | SPLoss:218.1046 | CLSLoss:0.4141 | top1:67.0000 | AUROC:0.8179
4/4 | Loss:23.8008 | MainLoss:0.4474 | SPLoss:233.4926 | CLSLoss:0.4137 | top1:70.5000 | AUROC:0.8400

Epoch: [135 | 4000] LR: 0.031925
1/4 | Loss:25.7342 | MainLoss:0.4505 | SPLoss:252.7957 | CLSLoss:0.4184 | top1:71.0000 | AUROC:0.8490
4/4 | Loss:26.1261 | MainLoss:0.4311 | SPLoss:256.9081 | CLSLoss:0.4216 | top1:74.2500 | AUROC:0.8683

Epoch: [136 | 4000] LR: 0.031924
1/4 | Loss:26.2013 | MainLoss:0.3772 | SPLoss:258.1993 | CLSLoss:0.4185 | top1:79.0000 | AUROC:0.8868
4/4 | Loss:25.7392 | MainLoss:0.3990 | SPLoss:253.3612 | CLSLoss:0.4112 | top1:75.7500 | AUROC:0.8690

Epoch: [137 | 4000] LR: 0.031923
1/4 | Loss:24.6328 | MainLoss:0.4420 | SPLoss:241.8678 | CLSLoss:0.4017 | top1:72.0000 | AUROC:0.8407
4/4 | Loss:23.5915 | MainLoss:0.4125 | SPLoss:231.7504 | CLSLoss:0.3995 | top1:73.2500 | AUROC:0.8567

Epoch: [138 | 4000] LR: 0.031922
1/4 | Loss:21.7130 | MainLoss:0.4011 | SPLoss:21

...

4/4 | Loss:2.3039 | MainLoss:0.3773 | SPLoss:19.2245 | CLSLoss:0.4154 | top1:75.5000 | AUROC:0.8776
39/4 | Loss:2.6904 | MainLoss:0.7946 | SPLoss:18.9164 | CLSLoss:0.4160 | top1:47.4615 | AUROC:0.4378
161/4 | Loss:1.9514 | MainLoss:0.0556 | SPLoss:18.9164 | CLSLoss:0.4160 | top1:97.9657 | AUROC:0.9990

Epoch: [202 | 4000] LR: 0.003182
1/4 | Loss:2.2402 | MainLoss:0.3444 | SPLoss:18.9164 | CLSLoss:0.4160 | top1:76.0000 | AUROC:0.8914
4/4 | Loss:2.2430 | MainLoss:0.3653 | SPLoss:18.7347 | CLSLoss:0.4164 | top1:74.0000 | AUROC:0.8721

Epoch: [203 | 4000] LR: 0.003182
1/4 | Loss:2.1690 | MainLoss:0.3214 | SPLoss:18.4346 | CLSLoss:0.4168 | top1:83.0000 | AUROC:0.9239
4/4 | Loss:2.2102 | MainLoss:0.3802 | SPLoss:18.2580 | CLSLoss:0.4170 | top1:75.0000 | AUROC:0.8759

Epoch: [204 | 4000] LR: 0.003182
1/4 | Loss:2.2280 | MainLoss:0.4272 | SPLoss:17.9664 | CLSLoss:0.4169 | top1:67.0000 | AUROC:0.8267
4/4 | Loss:2.1505 | MainLoss:0.3669 | SPLoss:17.7942 | CLSLoss:0.4168 | top1:76.7500 | AUROC:0.

1/4 | Loss:1.2214 | MainLoss:0.4009 | SPLoss:8.1617 | CLSLoss:0.4289 | top1:75.0000 | AUROC:0.8551
4/4 | Loss:1.1880 | MainLoss:0.3752 | SPLoss:8.0858 | CLSLoss:0.4290 | top1:76.0000 | AUROC:0.8727

Epoch: [236 | 4000] LR: 0.003175
1/4 | Loss:1.1930 | MainLoss:0.3927 | SPLoss:7.9603 | CLSLoss:0.4292 | top1:70.0000 | AUROC:0.8518
4/4 | Loss:1.1551 | MainLoss:0.3621 | SPLoss:7.8862 | CLSLoss:0.4294 | top1:74.0000 | AUROC:0.8702

Epoch: [237 | 4000] LR: 0.003175
1/4 | Loss:1.1702 | MainLoss:0.3895 | SPLoss:7.7638 | CLSLoss:0.4297 | top1:75.0000 | AUROC:0.8715
4/4 | Loss:1.1345 | MainLoss:0.3611 | SPLoss:7.6913 | CLSLoss:0.4298 | top1:77.5000 | AUROC:0.8910

Epoch: [238 | 4000] LR: 0.003175
1/4 | Loss:1.0836 | MainLoss:0.3222 | SPLoss:7.5717 | CLSLoss:0.4301 | top1:78.0000 | AUROC:0.9030
4/4 | Loss:1.1139 | MainLoss:0.3594 | SPLoss:7.5022 | CLSLoss:0.4302 | top1:73.7500 | AUROC:0.8734

Epoch: [239 | 4000] LR: 0.003175
1/4 | Loss:1.1220 | MainLoss:0.3789 | SPLoss:7.3880 | CLSLoss:0.4304 | t

1/4 | Loss:0.9026 | MainLoss:0.3573 | SPLoss:5.4107 | CLSLoss:0.4255 | top1:76.0000 | AUROC:0.8973
4/4 | Loss:0.9025 | MainLoss:0.3577 | SPLoss:5.4058 | CLSLoss:0.4255 | top1:77.5000 | AUROC:0.8940

Epoch: [270 | 4000] LR: 0.000317
1/4 | Loss:0.9025 | MainLoss:0.3585 | SPLoss:5.3975 | CLSLoss:0.4255 | top1:79.0000 | AUROC:0.8988
4/4 | Loss:0.9083 | MainLoss:0.3648 | SPLoss:5.3925 | CLSLoss:0.4256 | top1:77.5000 | AUROC:0.8883

Epoch: [271 | 4000] LR: 0.000317
1/4 | Loss:0.9453 | MainLoss:0.4026 | SPLoss:5.3843 | CLSLoss:0.4256 | top1:77.0000 | AUROC:0.8679
4/4 | Loss:0.9560 | MainLoss:0.4138 | SPLoss:5.3794 | CLSLoss:0.4256 | top1:75.0000 | AUROC:0.8580

Epoch: [272 | 4000] LR: 0.000317
1/4 | Loss:0.9179 | MainLoss:0.3765 | SPLoss:5.3713 | CLSLoss:0.4256 | top1:77.0000 | AUROC:0.8731
4/4 | Loss:0.9127 | MainLoss:0.3718 | SPLoss:5.3664 | CLSLoss:0.4256 | top1:76.2500 | AUROC:0.8772

Epoch: [273 | 4000] LR: 0.000317
1/4 | Loss:0.9172 | MainLoss:0.3771 | SPLoss:5.3583 | CLSLoss:0.4256 | t

1/4 | Loss:0.8038 | MainLoss:0.3013 | SPLoss:4.9817 | CLSLoss:0.4274 | top1:76.0000 | AUROC:0.9149
4/4 | Loss:0.8654 | MainLoss:0.3634 | SPLoss:4.9772 | CLSLoss:0.4274 | top1:74.7500 | AUROC:0.8731

Epoch: [304 | 4000] LR: 0.000316
1/4 | Loss:0.8770 | MainLoss:0.3758 | SPLoss:4.9698 | CLSLoss:0.4275 | top1:75.0000 | AUROC:0.8715
4/4 | Loss:0.8695 | MainLoss:0.3687 | SPLoss:4.9653 | CLSLoss:0.4275 | top1:76.5000 | AUROC:0.8856

Epoch: [305 | 4000] LR: 0.000316
1/4 | Loss:0.8712 | MainLoss:0.3712 | SPLoss:4.9579 | CLSLoss:0.4275 | top1:75.0000 | AUROC:0.8929
4/4 | Loss:0.8604 | MainLoss:0.3608 | SPLoss:4.9534 | CLSLoss:0.4275 | top1:77.2500 | AUROC:0.8893

Epoch: [306 | 4000] LR: 0.000316
1/4 | Loss:0.8727 | MainLoss:0.3738 | SPLoss:4.9459 | CLSLoss:0.4276 | top1:68.0000 | AUROC:0.8605
4/4 | Loss:0.8648 | MainLoss:0.3663 | SPLoss:4.9414 | CLSLoss:0.4276 | top1:74.7500 | AUROC:0.8788

Epoch: [307 | 4000] LR: 0.000316
1/4 | Loss:0.8636 | MainLoss:0.3659 | SPLoss:4.9340 | CLSLoss:0.4276 | t

1/4 | Loss:0.8730 | MainLoss:0.4108 | SPLoss:4.5792 | CLSLoss:0.4296 | top1:76.0000 | AUROC:0.8421
4/4 | Loss:0.8209 | MainLoss:0.3591 | SPLoss:4.5749 | CLSLoss:0.4296 | top1:76.2500 | AUROC:0.8796

Epoch: [339 | 4000] LR: 0.000315
1/4 | Loss:0.8616 | MainLoss:0.4005 | SPLoss:4.5679 | CLSLoss:0.4297 | top1:70.0000 | AUROC:0.8555
4/4 | Loss:0.8252 | MainLoss:0.3645 | SPLoss:4.5637 | CLSLoss:0.4298 | top1:75.5000 | AUROC:0.8923

Epoch: [340 | 4000] LR: 0.000315
1/4 | Loss:0.8017 | MainLoss:0.3417 | SPLoss:4.5567 | CLSLoss:0.4298 | top1:81.0000 | AUROC:0.9102
4/4 | Loss:0.8290 | MainLoss:0.3694 | SPLoss:4.5526 | CLSLoss:0.4298 | top1:76.0000 | AUROC:0.8753

Epoch: [341 | 4000] LR: 0.000315
1/4 | Loss:0.8527 | MainLoss:0.3938 | SPLoss:4.5457 | CLSLoss:0.4299 | top1:72.0000 | AUROC:0.8583
4/4 | Loss:0.8151 | MainLoss:0.3566 | SPLoss:4.5416 | CLSLoss:0.4299 | top1:76.5000 | AUROC:0.8896
39/4 | Loss:1.2018 | MainLoss:0.7440 | SPLoss:4.5348 | CLSLoss:0.4300 | top1:48.3846 | AUROC:0.4648
161/4 